### Pre-requisites

Install gensim using the following command

```pip install gensim```

Install or update NLTK module

```
- pip install --user -U nltk
- pip install --upgrade nltk
```

### Overview

***Reuters Corpus***

The Reuters Corpus contains 10,788 news documents totaling 1.3 million words. The documents have been classified into 90 topics, and grouped into two sets, called "training" and "test";

### Resources

- https://www.nltk.org/install.html
- https://www.nltk.org/data.html
- https://www.nltk.org/book/ch02.html

In [11]:
# Import libraries

import sys
assert sys.version_info[0] == 3
assert sys.version_info[1] >= 5
print(sys.version_info)

from gensim.models import KeyedVectors
from gensim.test.utils import datapath
import pprint
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]

# Download Reuters corpus from NLTK data distribution
import nltk
nltk.download('reuters')
from nltk.corpus import reuters

import numpy as np
import random
import scipy as sp
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA

START_TOKEN = '<START>'
END_TOKEN = '<END>'

np.random.seed(0)
random.seed(0)

sys.version_info(major=3, minor=7, micro=3, releaselevel='final', serial=0)


[nltk_data] Downloading package reuters to /home/agoel/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


### Exploring Reuters corpus

Unlike the Brown Corpus, categories in the Reuters corpus overlap with each other, simply because a news story often covers multiple topics. We can ask for the topics covered by one or more documents, or for the documents included in one or more categories. For convenience, the corpus methods accept a single fileid or a list of fileids.

In [43]:
# The text with fileid 'test/14826' is a document drawn from the test set.
print(reuters.fileids()[0:10])
# News document count
print('News documents count: {}'.format(len(reuters.fileids())))

['test/14826', 'test/14828', 'test/14829', 'test/14832', 'test/14833', 'test/14839', 'test/14840', 'test/14841', 'test/14842', 'test/14843']
News documents count: 10788


In [65]:
print('categories count: {}'.format(len(reuters.categories())), end='\n\n')
print('Categories: {}'.format(reuters.categories()[0:20]), end='\n\n')
print('Categories in the document \'training/9865\': {}'.format(reuters.categories('training/9865')), end='\n\n')
print(reuters.fileids('barley')[0:10], end='\n\n')
print('Words by document/file: {}'.format(reuters.words('training/9865')[:14]), end='\n\n')
print('Words by categories: {}'.format(reuters.words(categories='barley')), end='\n\n')
print('Words by multiple categories: {}'.format(reuters.words(categories=['barley', 'corn'])))

categories count: 90

Categories: ['acq', 'alum', 'barley', 'bop', 'carcass', 'castor-oil', 'cocoa', 'coconut', 'coconut-oil', 'coffee', 'copper', 'copra-cake', 'corn', 'cotton', 'cotton-oil', 'cpi', 'cpu', 'crude', 'dfl', 'dlr']

Categories in the document 'training/9865': ['barley', 'corn', 'grain', 'wheat']

['test/15618', 'test/15649', 'test/15676', 'test/15728', 'test/15871', 'test/15875', 'test/15952', 'test/17767', 'test/17769', 'test/18024']

Words by document/file: ['FRENCH', 'FREE', 'MARKET', 'CEREAL', 'EXPORT', 'BIDS', 'DETAILED', 'French', 'operators', 'have', 'requested', 'licences', 'to', 'export']

Words by categories: ['FRENCH', 'FREE', 'MARKET', 'CEREAL', 'EXPORT', ...]

Words by multiple categories: ['THAI', 'TRADE', 'DEFICIT', 'WIDENS', 'IN', 'FIRST', ...]


### Co-Occurrence

A co-occurrence matrix counts how often things co-occur in some environment. Given some word $w_i$  occurring in the document, we consider the context window surrounding $w_i$. Supposing our fixed window size is n, then this is the n preceding and n subsequent words in that document, i.e. words  $w_{i−n}…w_{i−1}$  and  $w_{i+1}…w_{i+n}$ . We build a co-occurrence matrix M, which is a symmetric word-by-word matrix in which  $M_{ij}$  is the number of times $w_{j}$ appears inside  $w_{i}$'s window.

***Example: Co-Occurrence with Fixed Window of n=1:***

Document 1: "all that glitters is not gold"

Document 2: "all is well that ends well"


The rows (or columns) of this matrix provide one type of word vectors (those based on word-word co-occurrence), but the vectors will be large in general (linear in the number of distinct words in a corpus). 

### Plotting Co-Occurrence Word Embeddings

In [13]:
def read_corpus(category='crude'):
    
    """ Read files from the specified Reuter's category.
        Params:
            category (string): category name
        Return:
            list of lists, with words from each of the processed files
    """
    
    files = reuters.fileids(category)
    return [[START_TOKEN] + [w.lower() for w in list(reuters.words(f))] + [END_TOKEN] for f in files]

In [74]:
reuters_corpus = read_corpus()
pprint.pprint(reuters_corpus[:3], compact=True, width=90)

[['<START>', 'japan', 'to', 'revise', 'long', '-', 'term', 'energy', 'demand',
  'downwards', 'the', 'ministry', 'of', 'international', 'trade', 'and', 'industry', '(',
  'miti', ')', 'will', 'revise', 'its', 'long', '-', 'term', 'energy', 'supply', '/',
  'demand', 'outlook', 'by', 'august', 'to', 'meet', 'a', 'forecast', 'downtrend', 'in',
  'japanese', 'energy', 'demand', ',', 'ministry', 'officials', 'said', '.', 'miti', 'is',
  'expected', 'to', 'lower', 'the', 'projection', 'for', 'primary', 'energy', 'supplies',
  'in', 'the', 'year', '2000', 'to', '550', 'mln', 'kilolitres', '(', 'kl', ')', 'from',
  '600', 'mln', ',', 'they', 'said', '.', 'the', 'decision', 'follows', 'the',
  'emergence', 'of', 'structural', 'changes', 'in', 'japanese', 'industry', 'following',
  'the', 'rise', 'in', 'the', 'value', 'of', 'the', 'yen', 'and', 'a', 'decline', 'in',
  'domestic', 'electric', 'power', 'demand', '.', 'miti', 'is', 'planning', 'to', 'work',
  'out', 'a', 'revised', 'energy', 'supp

### Find distinct words in a corpus

Below is a method to work out the distinct words (word types) that occur in the corpus. We can do this with for loops, but it's more efficient to do it with Python list comprehensions. In particular, please refer to [this](http://web.stanford.edu/class/cs224n/assignments/a1_preview/exploring_word_vectors.html) to flatten a list of lists. For more information on list comprehensions, please refer [here](https://coderwall.com/p/rcmaea/flatten-a-list-of-lists-in-one-line-in-python)

You may find it useful to use [Python sets](https://www.w3schools.com/python/python_sets.asp) to remove duplicate words.

In [146]:
def distinct_words(corpus):
    
    """ Determine a list of distinct words for the corpus.
        Params:
            corpus (list of list of strings): corpus of documents
        Return:
            corpus_words (list of strings): list of distinct words across the corpus, sorted (using python 'sorted' function)
            num_corpus_words (integer): number of distinct words across the corpus
    """
    corpus_words = []
    num_corpus_words = -1
    
    """List comprehensions are cleaner and faster than the for loop
       A set is a collection which is unordered, unindexed and contains unique elements. 
      In Python sets are written with curly brackets.
    """
    corpus_set = {w for s in corpus for w in s}
    sorted_corpus_set = sorted(corpus_set)
    corpus_words = sorted_corpus_set
    num_corpus_words = len(corpus_words)
    
    return corpus_words, num_corpus_words

In [148]:
# Define toy corpus
test_corpus = ["START All that glitters isn't gold END".split(" "), "START All's well that ends well END".split(" ")]
test_corpus_words, num_corpus_words = distinct_words(test_corpus)

# Correct answers
ans_test_corpus_words = sorted(list(set(["START", "All", "ends", "that", "gold", "All's", "glitters", "isn't", "well", "END"])))
ans_num_corpus_words = len(ans_test_corpus_words)

# Test correct number of words
assert(num_corpus_words == ans_num_corpus_words), "Incorrect number of distinct words. Correct: {}. Yours: {}".format(ans_num_corpus_words, num_corpus_words)

# Test correct words
assert (test_corpus_words == ans_test_corpus_words), "Incorrect corpus_words.\nCorrect: {}\nYours:   {}".format(str(ans_test_corpus_words), str(test_corpus_words))

# Print Success
print ("-" * 80)
print("Passed All Tests!")
print ("-" * 80)

--------------------------------------------------------------------------------
Passed All Tests!
--------------------------------------------------------------------------------


### Compute co-occurence matrix